# AD9081 Max Rate Utility Example

This example shows the use of the utility function **get_max_sample_rates** to determine the maximum same rates based on the number of converters (JESD param M) and limitations based on the FPGA itself.

For reference:
- L is the number of lanes used per link and 
- Sample clock is in samples per second
- Bit clock is equilavent to lane rate (bit clock is the technical name for lane rate)

In [ ]:
# This example determines the maximum sample rate based on
# FPGA platform and JESD204 class
import adijif as jif
import numpy as np
import pandas as pd
import ipywidgets as widgets
from IPython.display import display


In [ ]:
# Set up Converter and FPGA models

dd = widgets.Dropdown(
    options=["zc706", "zcu102", "vcu118"],
    value="zc706",
    description="FPGA Carrier:",
    disabled=False,
    interactive=True,
    layout={"width": "max-content"},
    style={"description_width": "initial"},
)

cb = widgets.Checkbox(
    value=False, description="Force QPLL", disabled=False, indent=False
)

Mmax = widgets.Dropdown(
    options=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16],
    value=16,
    description="M max:",
    disabled=False,
    interactive=True,
    layout={"width": "max-content"},
    style={"description_width": "initial"},
)

Mmin = widgets.Dropdown(
    options=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16],
    value=1,
    description="M min:",
    disabled=False,
    interactive=True,
    layout={"width": "max-content"},
    style={"description_width": "initial"},
)


def view_config(change, cb, Mmax, Mmin):
    conv = jif.ad9081_rx()

    fpga = jif.xilinx()
    fpga.setup_by_dev_kit_name(change)
    if cb:
        fpga.sys_clk_select = "GTH34_SYSCLK_QPLL0"  # Use faster QPLL

    limits = {"M": {">=": Mmin, "<=": Mmax}}

    results = jif.utils.get_max_sample_rates(conv, fpga, limits)

    # Print table
    out1 = widgets.Output()
    df1 = pd.DataFrame.from_dict(results)
    with out1:
        display(df1)
    h1 = widgets.HTML(value="<h2>Constrained</h2>")

    hb1 = widgets.VBox([h1, out1])

    results = jif.utils.get_max_sample_rates(conv)

    # Print table
    out2 = widgets.Output()
    df2 = pd.DataFrame.from_dict(results)
    with out2:
        display(df2)
    h2 = widgets.HTML(value="<h2>Unconstrained</h2>")

    hb2 = widgets.VBox([h2, out2])

    hh = widgets.HTML("   ")
    ui = widgets.HBox([hb1, hh, hb2])
    display(ui)


out = widgets.interactive_output(
    view_config, {"change": dd, "cb": cb, "Mmax": Mmax, "Mmin": Mmin}
)

heading = widgets.HTML(value="<h1>Configuration</h1>")
hr = widgets.HTML("<hr>")

FPGA = widgets.HBox([dd, cb])
M = widgets.HBox([Mmin, Mmax])

# Setup layout
tab = widgets.Tab()
children = [FPGA, M]
tab.children = children
for i, title in enumerate(["FPGA Setting", "JESD Settings"]):
    tab.set_title(i, title)

ui = widgets.VBox([heading, hr, tab, hr, out])
display(ui)